In [1]:
import pandas as pd
import re
import unicodedata

In [2]:
grados = pd.read_csv("../Output/grados.csv", sep=";")
grados_ofertados = pd.read_csv("../Output/grados_ofertados.csv", sep=";")
facultades = pd.read_csv("../Output/facultad.csv", sep=";")
universidades = pd.read_csv("../Output/universidad.csv", sep=";")
notas_corte = pd.read_csv("../Input/notas_corte.csv", sep=",")

In [3]:
def normalizar_texto(s):
    if pd.isna(s):
        return s
    s = str(s).strip().lower()
    s = " ".join(s.split())
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.replace("–", "-").replace("—", "-")
    return s

for df in [notas_corte, grados, universidades]:
    for col in ["grado"] if "grado" in df.columns else []:
        df[col] = df[col].apply(normalizar_texto)
    for col in ["universidad"] if "universidad" in df.columns else []:
        df[col] = df[col].apply(normalizar_texto)
    for col in ["nombre"] if "nombre" in df.columns else []:
        df[col] = df[col].apply(normalizar_texto)
    for col in ["nombre_uni"] if "nombre_uni" in df.columns else []:
        df[col] = df[col].apply(normalizar_texto)

ofertas = (
    grados_ofertados
    .merge(grados, left_on="id_grado", right_on="id", how="left", suffixes=("_of", "_gr"))
    .merge(facultades, left_on="id_facultad", right_on="id", how="left", suffixes=("", "_fac"))
    .merge(universidades, left_on="id_universidad", right_on="id", how="left", suffixes=("", "_uni"))
)

ofertas_simpl = ofertas[["id_of", "nombre", "nombre_uni"]].rename(
    columns={"nombre": "grado", "nombre_uni": "universidad"}
)

ofertas_simpl["grado"] = ofertas_simpl["grado"].apply(normalizar_texto)
ofertas_simpl["universidad"] = ofertas_simpl["universidad"].apply(normalizar_texto)

merged = notas_corte.merge(
    ofertas_simpl,
    on=["universidad", "grado"],
    how="left"
)

nota_corte = merged[["id_of", "año", "convocatoria", "grupo", "nota"]].rename(
    columns={"id_of": "id_grado_ofertado"}
)

nota_corte["nota"] = pd.to_numeric(nota_corte["nota"], errors="coerce")

nota_corte = (
    nota_corte
    .sort_values("nota", ascending=False)
    .drop_duplicates(subset=["id_grado_ofertado", "año", "convocatoria", "grupo"], keep="first")
    .reset_index(drop=True)
)

nota_corte = nota_corte.dropna(subset=["id_grado_ofertado"])
nota_corte["id_grado_ofertado"] = nota_corte["id_grado_ofertado"].astype(int)

nota_corte.insert(0, "id", range(1, len(nota_corte) + 1))

nota_corte.to_csv("../Output/nota_corte.csv", sep=";", index=False)